In [1]:
# data processing
import numpy as np
import pandas as pd
# pyplot
from matplotlib import pyplot as plt
# seaborn
import seaborn as sns
# bokeh
from bokeh import plotting as bk
from bokeh.models import Span
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource
import os
output_notebook()

Loading BokehJS ...

In [2]:
# Compound utils
from compound import (
    get_compound_return,
    simulate_compound_return,
    build_dataframe,
    MONTHS_IN_YEAR,
    define_scenario,
    plot_scenario_bokeh
)

In [3]:
sns.set_style('darkgrid')
sns.color_palette("Paired")
sns.set_palette("Paired")

In [14]:
step_amount = 5_000
max_amount = 20_000
initial_amounts = np.arange(step_amount,
                            max_amount + step_amount,
                            step_amount)
terms = [1, 3, 6, 12]
virtual_returns = np.arange(0.06, 0.12, 0.01).tolist()
yearly_contributions = list(map(lambda x: 12 * x, range(50, 500, 50)))
inc_contributions = [0.01, 0.03]
years_of_investment = [30]#np.arange(30, 30 + 5, 5)
inflation_rates = [0.02, 0.03, 0.04]#np.arange(30, 30 + 5, 5)
retirement_years = [5, 10, 15, 20]
monthly_retirement_incomes = [
    1.0e2,
    2.0e2,
    3.0e2,
    4.0e2,
    5.0e2
    ]
#tax_rate = [0.25]
tax_rate = ["spain"]
retirement_contribution_rate = [0.1]

results = define_scenario(initial_amounts,
                          virtual_returns,
                          years_of_investment,
                          terms,
                          yearly_contributions,
                          inc_contributions,
                          inflation_rates,
                          monthly_retirement_incomes,
                          retirement_years,
                          tax_rate
                          )

df = build_dataframe(results)
df = df[df["stable_yield"]]
df

,initial_amount,roi,duration,term,contribution,inc_contribution,inflation_rate,monthly_retirement_income,retirement_at,tax_rate,stable_yield,total_amount
123,5000,0.06,30,1,1200,0.01,0.02,100.0,20,spain,True,45148.117322
182,5000,0.06,30,1,1200,0.03,0.02,100.0,15,spain,True,32251.412986
183,5000,0.06,30,1,1200,0.03,0.02,100.0,20,spain,True,55294.544976
203,5000,0.06,30,1,1200,0.03,0.03,100.0,20,spain,True,42581.854610
242,5000,0.06,30,1,1800,0.01,0.02,100.0,15,spain,True,44874.929766
...,...,...,...,...,...,...,...,...,...,...,...,...
102593,20000,0.11,30,6,5400,0.03,0.04,400.0,10,spain,True,124094.408690
102594,20000,0.11,30,6,5400,0.03,0.04,400.0,15,spain,True,265903.247041
102595,20000,0.11,30,6,5400,0.03,0.04,400.0,20,spain,True,390305.711706
102598,20000,0.11,30,6,5400,0.03,0.04,500.0,15,spain,True,239804.892636


Scenarios that fulfill

1. Retirement years 5
1. Retirement income 500
1. Inflation rate 0.04

In [20]:
scenarios_1 = df.loc[
    (df["retirement_at"] == 5) & \
    (df["monthly_retirement_income"] == 500) & \
    (df["inflation_rate"] == 0.04)
]
scenarios_1

,initial_amount,roi,duration,term,contribution,inc_contribution,inflation_rate,monthly_retirement_income,retirement_at,tax_rate,stable_yield,total_amount


Scenarios that fulfill

1. Retirement years 10
1. Retirement income 500
1. Inflation rate 0.04

In [22]:
scenarios_2 = df.loc[
    (df["retirement_at"] == 10) & \
    (df["monthly_retirement_income"] == 500) & \
    (df["inflation_rate"] == 0.04)
]
scenarios_2

,initial_amount,roi,duration,term,contribution,inc_contribution,inflation_rate,monthly_retirement_income,retirement_at,tax_rate,stable_yield,total_amount
100437,20000,0.11,30,1,5400,0.03,0.04,500.0,10,spain,True,93758.458581
101517,20000,0.11,30,3,5400,0.03,0.04,500.0,10,spain,True,89961.213499


Scenarios that fulfill

1. Retirement years 10
1. Retirement income 400
1. Inflation rate 0.04

In [23]:
scenarios_3 = df.loc[
    (df["retirement_at"] == 10) & \
    (df["monthly_retirement_income"] == 400) & \
    (df["inflation_rate"] == 0.04)
]
scenarios_3

,initial_amount,roi,duration,term,contribution,inc_contribution,inflation_rate,monthly_retirement_income,retirement_at,tax_rate,stable_yield,total_amount
22673,5000,0.11,30,1,5400,0.03,0.04,400.0,10,spain,True,71085.517746
48473,10000,0.11,30,1,4800,0.03,0.04,400.0,10,spain,True,69254.358886
48533,10000,0.11,30,1,5400,0.01,0.04,400.0,10,spain,True,73256.989500
48593,10000,0.11,30,1,5400,0.03,0.04,400.0,10,spain,True,91887.766292
49613,10000,0.11,30,3,5400,0.01,0.04,400.0,10,spain,True,70485.323642
49673,10000,0.11,30,3,5400,0.03,0.04,400.0,10,spain,True,88938.544227
50753,10000,0.11,30,6,5400,0.03,0.04,400.0,10,spain,True,84725.680893
74273,15000,0.11,30,1,4200,0.03,0.04,400.0,10,spain,True,67413.377176
74333,15000,0.11,30,1,4800,0.01,0.04,400.0,10,spain,True,73483.564832
74393,15000,0.11,30,1,4800,0.03,0.04,400.0,10,spain,True,90044.255313


Scenarios that fulfill

1. Retirement years 10
1. Retirement income 300
1. Inflation rate 0.04

In [26]:
scenarios_4 = df.loc[
    (df["retirement_at"] == 10) & \
    (df["monthly_retirement_income"] == 350) & \
    (df["inflation_rate"] == 0.04)
]
scenarios_4

,initial_amount,roi,duration,term,contribution,inc_contribution,inflation_rate,monthly_retirement_income,retirement_at,tax_rate,stable_yield,total_amount
22549,5000,0.11,30,1,4800,0.03,0.04,350.0,10,spain,True,68244.422542
22609,5000,0.11,30,1,5400,0.01,0.04,350.0,10,spain,True,72259.142610
22669,5000,0.11,30,1,5400,0.03,0.04,350.0,10,spain,True,90908.000825
23629,5000,0.11,30,3,4800,0.03,0.04,350.0,10,spain,True,65962.335906
23689,5000,0.11,30,3,5400,0.01,0.04,350.0,10,spain,True,69913.346015
...,...,...,...,...,...,...,...,...,...,...,...,...
102349,20000,0.11,30,6,4200,0.03,0.04,350.0,10,spain,True,99831.123435
102409,20000,0.11,30,6,4800,0.01,0.04,350.0,10,spain,True,105682.531362
102469,20000,0.11,30,6,4800,0.03,0.04,350.0,10,spain,True,121850.977197
102529,20000,0.11,30,6,5400,0.01,0.04,350.0,10,spain,True,125681.105695


Scenarios that fulfill

1. Retirement years 10
1. Retirement income 200
1. Inflation rate 0.04

In [27]:
scenarios_5 = df.loc[
    (df["retirement_at"] == 10) & \
    (df["monthly_retirement_income"] == 200) & \
    (df["inflation_rate"] == 0.04)
]
scenarios_5

,initial_amount,roi,duration,term,contribution,inc_contribution,inflation_rate,monthly_retirement_income,retirement_at,tax_rate,stable_yield,total_amount
9705,5000,0.08,30,1,5400,0.03,0.04,200.0,10,spain,True,64916.925496
10785,5000,0.08,30,3,5400,0.03,0.04,200.0,10,spain,True,64254.931884
11865,5000,0.08,30,6,5400,0.03,0.04,200.0,10,spain,True,63288.158146
13785,5000,0.09,30,1,4200,0.03,0.04,200.0,10,spain,True,56599.420990
13845,5000,0.09,30,1,4800,0.01,0.04,200.0,10,spain,True,60547.844030
...,...,...,...,...,...,...,...,...,...,...,...,...
102345,20000,0.11,30,6,4200,0.03,0.04,200.0,10,spain,True,159159.719142
102405,20000,0.11,30,6,4800,0.01,0.04,200.0,10,spain,True,165011.127069
102465,20000,0.11,30,6,4800,0.03,0.04,200.0,10,spain,True,181179.572903
102525,20000,0.11,30,6,5400,0.01,0.04,200.0,10,spain,True,185009.701402


Scenarios that fulfill

1. Retirement years 10
1. Retirement income 100
1. Inflation rate 0.04

In [28]:
scenarios_6 = df.loc[
    (df["retirement_at"] == 10) & \
    (df["monthly_retirement_income"] == 100) & \
    (df["inflation_rate"] == 0.04)
]
scenarios_6

,initial_amount,roi,duration,term,contribution,inc_contribution,inflation_rate,monthly_retirement_income,retirement_at,tax_rate,stable_yield,total_amount
5021,5000,0.07,30,1,3600,0.03,0.04,100.0,10,spain,True,43213.046968
5081,5000,0.07,30,1,4200,0.01,0.04,100.0,10,spain,True,46656.463803
5141,5000,0.07,30,1,4200,0.03,0.04,100.0,10,spain,True,53769.688296
5201,5000,0.07,30,1,4800,0.01,0.04,100.0,10,spain,True,56196.930204
5261,5000,0.07,30,1,4800,0.03,0.04,100.0,10,spain,True,64326.329625
...,...,...,...,...,...,...,...,...,...,...,...,...
102341,20000,0.11,30,6,4200,0.03,0.04,100.0,10,spain,True,198712.116279
102401,20000,0.11,30,6,4800,0.01,0.04,100.0,10,spain,True,204563.524206
102461,20000,0.11,30,6,4800,0.03,0.04,100.0,10,spain,True,220731.970041
102521,20000,0.11,30,6,5400,0.01,0.04,100.0,10,spain,True,224562.098539


Scenarios that fulfill

1. Retirement years 10
1. Retirement income 500
1. Inflation rate 0.04